In [4]:
import polars as pl

# Load the Bronze layer for AAPL
df = pl.read_parquet("/Users/architmanek/Desktop/DataEngineering/momentum_pipeline/data/bronze/AAPL.parquet")

# Preview the shape and schema
print(df.shape)
print(df.schema)

# Look at the first few rows
df.head(10)

(2516, 6)
Schema({"('Date', '')": Datetime(time_unit='ns', time_zone=None), "('Close', 'AAPL')": Float64, "('High', 'AAPL')": Float64, "('Low', 'AAPL')": Float64, "('Open', 'AAPL')": Float64, "('Volume', 'AAPL')": Int64})


"('Date', '')","('Close', 'AAPL')","('High', 'AAPL')","('Low', 'AAPL')","('Open', 'AAPL')","('Volume', 'AAPL')"
datetime[ns],f64,f64,f64,f64,i64
2015-01-02 00:00:00,24.288578,24.757332,23.848704,24.746224,212818400
2015-01-05 00:00:00,23.604332,24.137513,23.41772,24.057535,257142000
2015-01-06 00:00:00,23.606558,23.866483,23.244438,23.668762,263188400
2015-01-07 00:00:00,23.937571,24.037541,23.704304,23.815383,160423600
2015-01-08 00:00:00,24.857309,24.915071,24.148623,24.266369,237458000
2015-01-09 00:00:00,24.883968,25.159444,24.484082,25.030592,214798000
2015-01-12 00:00:00,24.270803,25.021698,24.170833,25.015034,198603200
2015-01-13 00:00:00,24.4863,25.059469,24.195273,24.755111,268367600
2015-01-14 00:00:00,24.392996,24.546284,24.104189,24.224155,195826400
